# preparing for N.N
* https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/

In [1]:
import numpy  as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.callbacks import TensorBoard

from time import time

Using TensorFlow backend.


In [2]:
# define documents

docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']

# can also work on 2D text               :)  :D  :) :D
'''
docs = [['best', 'laptop', 'machine', 'learning', 'artificial', 'intelligence'],
 ['rise', 'analysis', 'platforms', 'across', 'every', 'sector', 'leading', 'big', 'tech', 'companies', 'startups', 'integrating', 'machine', 'learning', 'deep', 'learning', 'existing', 'system'], 
 ['advancement', 'led', 'major', 'revolution', 'hardware', 'required', 'setups']]
'''
# define class labels
labels = np.array([1,1,1,1,1,0,0,0,0,0])

In [3]:
tokenizer =Tokenizer()#better to give certain vocab size to work with
tokenizer.fit_on_texts(docs)
doc_to_seq = tokenizer.texts_to_sequences(docs)

In [4]:
doc_to_seq

[[6, 2],
 [3, 1],
 [7, 4],
 [8, 1],
 [9],
 [10],
 [5, 4],
 [11, 3],
 [5, 1],
 [12, 13, 2, 14]]

In [7]:
for word, i in tokenizer.word_index.items():
    print(word+'  '+str(i))

work  1
done  2
good  3
effort  4
poor  5
well  6
great  7
nice  8
excellent  9
weak  10
not  11
could  12
have  13
better  14


# padding

In [5]:
pad_doc_to_seq = pad_sequences(doc_to_seq,maxlen=4,padding='post')

In [6]:
pad_doc_to_seq

array([[ 6,  2,  0,  0],
       [ 3,  1,  0,  0],
       [ 7,  4,  0,  0],
       [ 8,  1,  0,  0],
       [ 9,  0,  0,  0],
       [10,  0,  0,  0],
       [ 5,  4,  0,  0],
       [11,  3,  0,  0],
       [ 5,  1,  0,  0],
       [12, 13,  2, 14]])

# Importing pre trained Glove
* https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html

In [7]:
Glove_Embedding = dict()

f = open('./dataset/glove.6B.50d.txt', encoding="utf8")

for line in f:
	values = line.split() # split by lines
	word = values[0]# get 1st word in the line i.e the "word" itself
	coefs = np.asarray(values[1:], dtype='float32') # the preceding  is the vector itself
	Glove_Embedding[word] = coefs# its a dict of word(key) and vector(value)
f.close()

print('Loaded %s word vectors.' % len(Glove_Embedding))

Loaded 400000 word vectors.


#  seperate Embedding matrix for N.N with only tokens seen in corpus 
/ since 0 is reserved we add 1

In [8]:
VOCAB_SIZE = len(tokenizer.word_index.items())
VOCAB_SIZE = VOCAB_SIZE + 1
print(VOCAB_SIZE)

15


# USed Glove (already trained) for our task

In [9]:
#so we have to create 2d array
# 50 since we have loaded 50 gims Glove matrix
embedding_matrix = np.zeros((VOCAB_SIZE,50))
words_not_found = []   # words not found in Glove emb. w.r.t our data corpous

for word, i in tokenizer.word_index.items():
    
    glove_vector_reprsentation = Glove_Embedding[word]
    
    if glove_vector_reprsentation is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = glove_vector_reprsentation
    else:
        #storing wrds for later analysis
        words_not_found = word
        

# Handling OOV wordds

In [ ]:
for word in words_not_found:
    thr = model.wv.most_similar(word)
    if thr > 0.48:
        OOV[word] = model['word']
    else:
        OOV['NaN'] = 'NaN'

# also manually check worst substitution we are going to make are really good enough
        
        
#plot zipfs law
#it require vocab word count in corpous   
    

In [10]:
print(embedding_matrix.shape)
print(Glove_Embedding.keys)

(15, 50)
<built-in method keys of dict object at 0x0000000004E33AB0>


# keras Emb layer

In [11]:
embedding_layer = Embedding(VOCAB_SIZE,
                            50,
                            weights=[embedding_matrix],
                            input_length=4,
                            trainable=False)


In [12]:
model =  Sequential()
model.add(embedding_layer)
model.add(Flatten())
model.add(Dense(1, activation="sigmoid"))

#compil model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

print(model.summary())

ImportError: You need the TensorFlow module installed to use TensorBoard.

In [29]:
model.fit(pad_doc_to_seq, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(pad_doc_to_seq, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 89.999998
